# Baseline MLP with Arrhenius Kinetics and Symmetry TTA

This notebook implements the baseline approach following the strategy:
1. Spange descriptors for solvent features
2. Arrhenius kinetics features (1/T, ln(time), interaction)
3. MLP with BatchNorm, Dropout, HuberLoss
4. Symmetry TTA for mixed solvents
5. Bagging ensemble with 7 models

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from abc import ABC
import tqdm
import warnings
warnings.filterwarnings('ignore')

# Check GPU
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

torch.set_default_dtype(torch.double)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

GPU available: True
GPU: NVIDIA H100 80GB HBM3
Memory: 85.0 GB
Using device: cuda


In [2]:
# Data paths
DATA_DIR = '/home/data'

# Load data functions
def load_data(name="full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_DIR}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_DIR}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def load_features(name='spange_descriptors'):
    return pd.read_csv(f'{DATA_DIR}/{name}_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load Spange descriptors
SPANGE_DF = load_features('spange_descriptors')
print(f"Spange descriptors shape: {SPANGE_DF.shape}")

Spange descriptors shape: (26, 13)


In [ ]:
# Kinetic Featurizer with Arrhenius features
class KineticMixingFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.featurizer = SPANGE_DF
        # Features: rt, temp, inv_temp, log_time, interaction + spange (13)
        self.feats_dim = 5 + self.featurizer.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[["Residence Time", "Temperature"]].values.astype(np.float64)
        
        # Arrhenius kinetics features
        temp_c = X_vals[:, 1:2]  # Temperature in Celsius
        time_m = X_vals[:, 0:1]  # Residence time in minutes
        
        temp_k = temp_c + 273.15  # Convert to Kelvin
        inv_temp = 1000.0 / temp_k  # Inverse temperature (scaled)
        log_time = np.log(time_m + 1e-6)  # Log of time
        interaction = inv_temp * log_time  # Interaction term
        
        X_kinetic = torch.tensor(np.hstack([X_vals, inv_temp, log_time, interaction]))
        
        # Chemical features with linear mixing
        if self.mixed:
            A = torch.tensor(self.featurizer.loc[X["SOLVENT A NAME"]].values)
            B = torch.tensor(self.featurizer.loc[X["SOLVENT B NAME"]].values)
            pct = torch.tensor(X["SolventB%"].values.reshape(-1, 1))
            
            if flip:
                # Symmetry flip: swap A and B
                X_chem = B * (1 - (1-pct)) + A * (1-pct)
            else:
                X_chem = A * (1 - pct) + B * pct
        else:
            X_chem = torch.tensor(self.featurizer.loc[X["SOLVENT NAME"]].values)
            
        return torch.cat([X_kinetic, X_chem], dim=1)

In [ ]:
# MLP Architecture
class MLPInternal(nn.Module):
    def __init__(self, input_dim):
        super(MLPInternal, self).__init__()
        self.net = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(64, 3),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
# Symmetric Bagged Model
class SymmetricBaggedModel(nn.Module):
    def __init__(self, data='single', n_models=7, epochs=300):
        super().__init__()
        self.data_type = data
        self.featurizer = KineticMixingFeaturizer(mixed=(data=='full'))
        self.n_models = n_models
        self.epochs = epochs
        self.models = nn.ModuleList()

    def train_model(self, X_train, y_train, verbose=False):
        # Standard data
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            # Augmented data (train on both symmetries)
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all = X_std
            y_all = y_vals
            
        input_dim = X_all.shape[1]
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i * 100)  # Different seed for each model
            model = MLPInternal(input_dim).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=32, shuffle=True)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)
            criterion = nn.HuberLoss()
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='min', factor=0.5, patience=20
            )
            
            for epoch in range(self.epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        if len(self.models) == 0:
            raise ValueError("Model not trained yet")
            
        device = next(self.models[0].parameters()).device
        
        # Test Time Augmentation for mixed solvents
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X, flip=False).to(device)
            X_flip = self.featurizer.featurize(X, flip=True).to(device)
            
            pred_sum = torch.zeros((len(X), 3)).to(device)
            
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    p1 = model(X_std)
                    p2 = model(X_flip)
                    pred_sum += (p1 + p2) * 0.5
            
            avg_pred = pred_sum / self.n_models
        else:
            X_std = self.featurizer.featurize(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            avg_pred = pred_sum / self.n_models

        return avg_pred.cpu()

In [ ]:
# Task 0: Single Solvent (Leave-One-Solvent-Out CV)
print("--- TASK 0: Single Solvent ---")
X, Y = load_data("single_solvent")
print(f"Data shape: {X.shape}, Targets: {Y.shape}")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
all_actuals = []
fold_errors = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = SymmetricBaggedModel(data='single', n_models=7, epochs=300)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X).numpy()
    
    # Calculate fold error
    actuals = test_Y.values
    fold_mae = np.mean(np.abs(predictions - actuals))
    fold_errors.append(fold_mae)
    
    for row_idx, row in enumerate(predictions):
        all_predictions.append({
            "task": 0, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })
    all_actuals.extend(actuals.tolist())

submission_single_solvent = pd.DataFrame(all_predictions)
print(f"\nTask 0 - Mean MAE across folds: {np.mean(fold_errors):.5f} ± {np.std(fold_errors):.5f}")

In [ ]:
# Task 1: Full Data with Mixtures (Leave-One-Ramp-Out CV)
print("\n--- TASK 1: Full Data (Mixtures) ---")
X, Y = load_data("full")
print(f"Data shape: {X.shape}, Targets: {Y.shape}")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions_full = []
all_actuals_full = []
fold_errors_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = SymmetricBaggedModel(data='full', n_models=7, epochs=300)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X).numpy()
    
    # Calculate fold error
    actuals = test_Y.values
    fold_mae = np.mean(np.abs(predictions - actuals))
    fold_errors_full.append(fold_mae)
    
    for row_idx, row in enumerate(predictions):
        all_predictions_full.append({
            "task": 1, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]
        })
    all_actuals_full.extend(actuals.tolist())

submission_full_data = pd.DataFrame(all_predictions_full)
print(f"\nTask 1 - Mean MAE across folds: {np.mean(fold_errors_full):.5f} ± {np.std(fold_errors_full):.5f}")

In [ ]:
# Combine and save submission
submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"

# Save submission
submission.to_csv("/home/submission/submission.csv", index=True)
print(f"\nSubmission saved with {len(submission)} rows")

# Overall score calculation
overall_mae = (np.mean(fold_errors) * len(all_actuals) + np.mean(fold_errors_full) * len(all_actuals_full)) / (len(all_actuals) + len(all_actuals_full))
print(f"\n=== OVERALL ESTIMATED SCORE (MAE) ===")
print(f"Task 0 MAE: {np.mean(fold_errors):.5f}")
print(f"Task 1 MAE: {np.mean(fold_errors_full):.5f}")
print(f"Weighted Overall MAE: {overall_mae:.5f}")